# 🎓 Workshop: Building a Modern Data Warehouse with BigQuery & dbt

Welcome to this advanced hands-on lab. Today, we will transform raw e-commerce data into a professional **Star Schema** Data Warehouse using the **ELT (Extract, Load, Transform)** pattern.

---

### 🚀 Learning Objectives
By the end of this session, you will master the end-to-end Data Engineering lifecycle:

1.  **☁️ Cloud Setup & Infrastructure:**
    *   Authenticate with **Google Cloud Platform (GCP)**.
    *   Set up a serverless development environment in Colab.
    
2.  **📥 Data Ingestion (Extract & Load):**
    *   Automate downloading the **Brazilian E-Commerce Dataset** (100k+ orders) using the Kaggle API.
    *   Build a **Smart Uploader** in Python to sanitize and load raw CSVs into **BigQuery** (`ecommerce_raw`).

3.  **🔧 Code Refactoring & Automation:**
    *   Learn how to programmatically refactor a legacy **dbt (Data Build Tool)** project.
    *   Use Python scripting to update Project IDs and fix SQL syntax errors across dozens of files instantly.

4.  **🏗️ Analytics Engineering (Transform):**
    *   Execute a **dbt pipeline** to create:
        *   **🥈 Staging Layer:** Cleaned and standardized views/tables.
        *   **🥇 Marts Layer:** Business-ready Fact and Dimension tables (Star Schema).

---
### 🛠️ Tech Stack
*   **Language:** Python 3.10+
*   **Warehouse:** Google BigQuery (Standard SQL)
*   **Transformation:** dbt (Data Build Tool) Core 1.x
*   **Source:** Kaggle API

---
### ⚠️ Before We Begin
*   Ensure you have a valid **Google Cloud Project ID** ready.
*   The code below uses automated scripts to fix common configuration errors—run the cells sequentially.

*Let's build a production-grade pipeline!* 👇

In [ ]:
# 1. Google Cloud Authentication
# Authenticate the user with Google Cloud Platform to allow access to BigQuery and other GCP services.
from google.colab import auth
auth.authenticate_user()

# 2. Install Required Tools
# Install necessary Python libraries including dbt-bigquery adapter, opendatasets, and pandas-gbq for data manipulation and BigQuery interaction.
!pip install -q dbt-bigquery opendatasets pandas-gbq

# 3. Project Configuration
# Set the Google Cloud Project ID. Ensure this matches your specific GCP project identifier where BigQuery resources will be provisioned.
MY_PROJECT_ID = "astute-backup-394812"

# 4. Clone Repository
# Clone the project repository from GitHub if the directory does not already exist. This pulls the latest dbt project structure.
import os
if not os.path.exists('data_engineering_projects'):
    !git clone https://github.com/GhamdanDev/modern-data-warehouse-bq-dbt.git

# 5. Navigate to Project Directory
# Change the current working directory to the dbt project folder to execute dbt commands.
%cd /content/modern-data-warehouse-bq-dbt/GCP-COLAB-DBT/pipeline-ecommerce-bq-dbt/cloud_run_dbt/dbt

print("✅ Environment setup and authentication completed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.4/114.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.9/144.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.7/442.7 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import json
import pandas as pd
from google.cloud import bigquery

# 1. Kaggle Authentication Setup
# Programmatically configure Kaggle credentials to automate dataset downloading. 
# This prevents manual username/key prompts during execution by creating the .kaggle directory 
# and writing the API token securely.
kaggle_config = {"username":"ghamdannashwan","key":"KGAT_148e98e849742fbd54519ad67a04d107"}
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(kaggle_config, f)
!chmod 600 /root/.kaggle/kaggle.json

# 2. Download Dataset
# Download the Brazilian E-Commerce dataset directly using the Kaggle CLI for speed and reliability.
# The --unzip flag automatically extracts content to the specified path.
print("⏳ Downloading data from Kaggle...")
!kaggle datasets download -d olistbr/brazilian-ecommerce --unzip -p /content/brazilian-ecommerce

# 3. BigQuery Configuration
# Define project constants including the Project ID, BigQuery Dataset ID (Data Warehouse Schema), 
# and the local path where raw data is stored.
MY_PROJECT_ID = "astute-backup-394812"
DATASET_ID = "ecommerce_raw"
DATA_PATH = '/content/brazilian-ecommerce'

# File-to-Table Mapping
# Define a mapping dictionary to link raw CSV filenames to their corresponding target table names in BigQuery.
# This ensures standardized naming for the raw layer tables.
mapping_standard = {
    'olist_customers_dataset.csv': 'olist_customers_dataset',
    'olist_orders_dataset.csv': 'olist_orders_dataset',
    'olist_products_dataset.csv': 'olist_products_dataset',
    'olist_sellers_dataset.csv': 'olist_sellers_dataset',
    'olist_order_items_dataset.csv': 'olist_order_items_dataset',
    'olist_order_payments_dataset.csv': 'olist_order_payments_dataset',
    'olist_geolocation_dataset.csv': 'olist_geolocation_dataset'
}

client = bigquery.Client(project=MY_PROJECT_ID)

# 4. Dataset Validation
# Check if the target BigQuery dataset exists. If not, create it in the 'US' location 
# to ensure subsequent data loads succeed.
dataset_ref = client.dataset(DATASET_ID)
try:
    client.get_dataset(dataset_ref)
    print(f"✅ Dataset '{DATASET_ID}' already exists.")
except:
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"
    client.create_dataset(dataset)
    print(f"✨ Created new Dataset named '{DATASET_ID}'.")

# 5. Smart Data Ingestion
# Iterate through the file mapping to ingest CSVs into BigQuery using pandas-gbq.
# This process includes reading, cleaning, and uploading the data.
print("\n🚀 Starting upload process to BigQuery...")

for file_name, table_name in mapping_standard.items():
    full_path = os.path.join(DATA_PATH, file_name)

    if os.path.exists(full_path):
        try:
            # Read CSV with memory optimization (low_memory=False) to handle mixed types efficiently.
            df = pd.read_csv(full_path, low_memory=False)

            # Sanitize column names by replacing spaces and periods with underscores 
            # to comply with BigQuery naming conventions.
            df.columns = [c.replace(' ', '_').replace('.', '_') for c in df.columns]

            print(f"⏳ Uploading {table_name} ({len(df)} records)...")

            # Upload DataFrame to BigQuery using the `to_gbq` method.
            # 'if_exists=replace' overwrites the table if it already exists.
            df.to_gbq(
                destination_table=f"{DATASET_ID}.{table_name}",
                project_id=MY_PROJECT_ID,
                if_exists='replace',
                progress_bar=False,
                location='US'
            )
            print(f"✅ Successfully uploaded table: {table_name}")

        except Exception as e:
            print(f"❌ Failed to upload {table_name}. Error: {e}")
    else:
        print(f"⚠️ Warning: File {file_name} not found in the specified path.")

print("\n🎯 Task completed! All raw tables are now ready for transformation using dbt.")

⏳ جاري تحميل البيانات من Kaggle...
Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
License(s): CC-BY-NC-SA-4.0
  0% 0.00/42.6M [00:00<?, ?B/s]
100% 42.6M/42.6M [00:00<00:00, 1.46GB/s]
✨ تم إنشاء Dataset جديد باسم 'ecommerce_raw'.

🚀 بدء عملية الرفع إلى BigQuery...
⏳ جاري رفع olist_customers_dataset (99441 سجل)...


/tmp/ipython-input-2668302334.py:64: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(


✅ نجح رفع الجدول: olist_customers_dataset
⏳ جاري رفع olist_orders_dataset (99441 سجل)...


/tmp/ipython-input-2668302334.py:64: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(


✅ نجح رفع الجدول: olist_orders_dataset
⏳ جاري رفع olist_products_dataset (32951 سجل)...


...

✅ نجح رفع الجدول: olist_order_items_dataset
⏳ جاري رفع olist_order_payments_dataset (103886 سجل)...


/tmp/ipython-input-2668302334.py:64: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(


✅ نجح رفع الجدول: olist_order_payments_dataset
⏳ جاري رفع olist_geolocation_dataset (1000163 سجل)...


/tmp/ipython-input-2668302334.py:64: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(


✅ نجح رفع الجدول: olist_geolocation_dataset

🎯 انتهت المهمة! كافة الجداول الخام جاهزة الآن للتحويل باستخدام dbt.


In [ ]:
import os
import re

# 1. Refactoring Configuration
# Set up paths and project IDs to update the cloned dbt project with your specific GCP configuration.
# This ensures that all dbt models point to the correct Google Cloud Project.
DBT_PATH = "/content/modern-data-warehouse-bq-dbt/GCP-COLAB-DBT/pipeline-ecommerce-bq-dbt/cloud_run_dbt/dbt"
OLD_ID = "ecommerce-analysis-455200"
NEW_ID = MY_PROJECT_ID # astute-backup-394812

print(f"🚀 Starting comprehensive refactoring: from {OLD_ID} to {NEW_ID}")

# 2. Comprehensive Project Refactoring Function
# Define a robust function to walk through the project directory and replace hardcoded Project IDs 
# and clean up complex incremental logic that might fail in a sandbox environment.
def comprehensive_refactor(base_path):
    for root, dirs, files in os.walk(base_path):
        # Skip internal dbt system directories (dbt_packages, logs, target) to avoid corrupting build artifacts.
        if 'dbt_packages' in root or 'logs' in root or 'target' in root:
            continue

        for file in files:
            if file.endswith(('.yml', '.sql', '.md', '.json')):
                file_path = os.path.join(root, file)

                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()

                if OLD_ID in content or "materialized='incremental'" in content or "materialized: incremental" in content:
                    # A. Update Project ID: Replace the old hardcoded Project ID with the current user's Project ID.
                    new_content = content.replace(OLD_ID, NEW_ID)

                    # B. Materialization Strategy: Switch from 'incremental' to 'table' materialization 
                    # to simplify the workflow and avoid complexity in this sandbox environment.
                    new_content = new_content.replace("materialized='incremental'", "materialized='table'")
                    new_content = new_content.replace("materialized: incremental", "materialized: table")

                    # C. Code Cleanup: Remove complex jinja blocks associated with incremental logic 
                    # (like is_incremental()) to ensure cleaner execution.
                    new_content = re.sub(r"{% if is_incremental\(\) %}.*?{% endif %}", "", new_content, flags=re.DOTALL)

                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.write(new_content)
                    print(f"✅ Updated and refactored: {file}")

# 3. Execute Refactoring
# Run the refactoring function across the entire dbt project directory.
comprehensive_refactor(DBT_PATH)

# 4. Fix Project Configuration (dbt_project.yml)
# Specifically target `dbt_project.yml` (the heart of the project) to ensure global materialization settings 
# are correct and remove any lingering deprecated incremental configurations.
project_conf_path = os.path.join(DBT_PATH, "dbt_project.yml")
if os.path.exists(project_conf_path):
    with open(project_conf_path, 'r') as f:
        conf = f.read()
    # Ensure explicit table materialization is set to avoid deprecation warnings.
    conf = conf.replace("materialized: table", "materialized: table")
    # regex substitution to guarantee incremental logic is removed from models config.
    conf = re.sub(r"materialized: incremental", "materialized: table", conf)
    with open(project_conf_path, 'w') as f:
        f.write(conf)
    print("🎯 Fully repaired dbt_project.yml.")

# 5. Fix dim_orders Model
# Overwrite the `dim_orders.sql` model with a standard BigQuery SQL definition 
# to fix casting and timestamp issues, ensuring it uses Standard SQL dialect.
orders_model_path = os.path.join(DBT_PATH, "models/marts/dim_orders.sql")
final_sql = """
{{ config(materialized='table') }}
SELECT
    order_id, customer_id, order_status,
    SAFE_CAST(order_purchase_timestamp AS TIMESTAMP) AS order_purchase_timestamp,
    SAFE_CAST(order_approved_at AS TIMESTAMP) AS order_approved_at,
    SAFE_CAST(order_delivered_carrier_date AS TIMESTAMP) AS order_delivered_carrier_date,
    SAFE_CAST(order_delivered_customer_date AS TIMESTAMP) AS order_delivered_customer_date,
    SAFE_CAST(order_estimated_delivery_date AS TIMESTAMP) AS order_estimated_delivery_date,
    TIMESTAMP_TRUNC(SAFE_CAST(order_purchase_timestamp AS TIMESTAMP), DAY) AS order_purchase_date
FROM {{ ref('dim_orders_stg') }}
"""
with open(orders_model_path, 'w') as f:
    f.write(final_sql)

print("\n✨ dbt project is now clean, connected to your project, and ready for execution!")

🚀 بدء عملية التطهير الشامل: من ecommerce-analysis-455200 إلى astute-backup-394812
✅ تم تحديث وتطهير: profiles.yml
✅ تم تحديث وتطهير: dim_orders.sql
✅ تم تحديث وتطهير: dim_sellers.sql
✅ تم تحديث وتطهير: fact_order_items.sql
✅ تم تحديث وتطهير: dim_products.sql
✅ تم تحديث وتطهير: fact_order_payments.sql
✅ تم تحديث وتطهير: dim_customers.sql
✅ تم تحديث وتطهير: dim_products_stg.sql
✅ تم تحديث وتطهير: dim_sellers_stg.sql
✅ تم تحديث وتطهير: fact_order_items_stg.sql
✅ تم تحديث وتطهير: fact_order_payments_stg.sql
✅ تم تحديث وتطهير: dim_customers_stg.sql
✅ تم تحديث وتطهير: dim_orders_stg.sql
🎯 تم إصلاح ملف dbt_project.yml بالكامل.

✨ الآن dbt نظيف تماماً ومرتبط بمشروعك فقط. جاهز للتنفيذ!


# 📘 Data Engineering Architecture & Concepts Explained

Before executing the pipeline, it is crucial to understand the architectural decisions and data engineering concepts powering this project. This Modern Data Stack uses an **ELT (Extract, Load, Transform)** pattern, leveraging the power of BigQuery for processing.

### 1. The Architecture Layers (Medallion Architecture)

We organizes data into three distinct processing layers to ensure quality and usability:

*   **🥉 Raw Layer (Bronze):** 
    *   **Source:** Kaggle CSV files.
    *   **Action:** Direct ingestion into BigQuery (`ecommerce_raw` dataset).
    *   **Characteristics:** Data is "as-is", with potential duplicates or data type issues. We perform minimal changes here (just column name sanitization).

*   **🥈 Staging Layer (Silver):** (`models/stg/`)
    *   **Purpose:** Cleaning and Standardization.
    *   **Key Techniques Used:**
        *   **Deduplication:** Using SQL window functions (e.g., `QUALIFY COUNT(...) OVER(...)`) to ensure uniqueness, as seen in `dim_customers_stg` to resolve zip code conflicts.
        *   **Type Casting:** Converting strings to Dates/Timestamps using `SAFE_CAST` to prevent pipeline failures on bad data.
        *   **Renaming:** Changing technical column names to business-friendly terms.

*   **🥇 Marts Layer (Gold):** (`models/marts/`)
    *   **Purpose:** Business Analytics & Reporting.
    *   **Design Pattern:** **Star Schema**.
    *   **Components:**
        *   **Facts (`fact_`):** Operational metrics (e.g., `fact_order_items`). These tables contain foreign keys and numeric measures (Price, Freight Value).
        *   **Dimensions (`dim_`):** Contextual data (e.g., `dim_customers`, `dim_products`). These answer the "Who", "What", and "Where" questions.

### 2. Deep Dive: Dimensional Modeling (Star Schema)

The core of this warehouse allows for efficient querying by BI tools (like Looker or Tableau).

*   **Center:** `fact_order_items` (The transaction).
    *   *Links to:* `dim_customers` (Who bought it?)
    *   *Links to:* `dim_products` (What was bought?)
    *   *Links to:* `dim_sellers` (Who sold it?)
    *   *Links to:* `dim_orders` (When and how was it delivered?)

### 3. dbt (Data Build Tool) Concepts

*   **Materialization (`table` vs `incremental`):**
    *   You might notice our refactoring script forced `materialized='table'`. 
    *   **Incremental:** Best for massive datasets (PB scale). It only processes *new* data since the last run. It requires complex logic (`is_incremental()`) and unique keys.
    *   **Table:** Rebuilds the entire table from scratch every run. For this scale (Brazilian E-Commerce dataset), `table` is simpler, less error-prone, and faster to debug in a development environment.
    
*   **Ref (`{{ ref('...') }}`):** 
    *   This is dbt's magic. Instead of hardcoding table names like `project.dataset.table`, we use `ref()`. This builds a **Dependency Graph (DAG)**, ensuring `dim_customers_stg` runs *before* `dim_customers`.

### 4. Data Quality Checks

*   **Generic Tests:** dbt checks for `unique` and `not_null` constraints automatically defined in `schema.yml`.
*   **Custom Logic:** Our SQL transformations handle edge cases, such as `TIMESTAMP_TRUNC` in `dim_orders` to standardize purchase dates for daily aggregations.

---
*Ready to Build: The next cell executes `dbt run`, which compiles these SQL models into executable BigQuery code and runs them in the correct dependency order.*

In [ ]:
# 6. Execute dbt Pipeline transformation
# This is the final execution step where the transformations defined in the SQL models are applied.

# 1. Change Working Directory
# We strictly navigate to the `dbt` subdirectory. 
# Why? The `dbt run` command looks for `dbt_project.yml` in the current folder. 
# Running it from outside this directory would result in a "project not found" error.
%cd /content/modern-data-warehouse-bq-dbt/GCP-COLAB-DBT/pipeline-ecommerce-bq-dbt/cloud_run_dbt/dbt

# 2. Trigger Transformation (dbt run)
# The `dbt run` command performs the following actions:
#   1. Compiles your Jinja SQL files (from models/) into standard SQL.
#   2. Constructs the Dependency Graph (DAG) to determine the correct order (Staging -> Marts).
#   3. Executes the DDL/DML statements one by one in BigQuery.
#
# Expected Output:
#   - distinct "OK created table" messages for each model (dim_customers, fact_orders, etc.).
#   - If any model fails, the pipeline stops and provides an error log.
!dbt run

/content/modern-data-warehouse-bq-dbt/GCP-COLAB-DBT/pipeline-ecommerce-bq-dbt/cloud_run_dbt/dbt
22:31:21  Running with dbt=1.11.2
22:31:29  [WARNING][MissingPlusPrefixDeprecation]: Deprecated functionality
Missing '+' prefix on `materialized` found at
`pipeline_ecommerce.stg.materialized` in file `/content/modern-data-warehouse-bq
-dbt/GCP-COLAB-DBT/pipeline-ecommerce-bq-dbt/cloud_run_dbt/dbt/dbt_project.yml`.
Hierarchical config values without a '+' prefix are deprecated in
dbt_project.yml.
22:31:30  Registered adapter: bigquery=1.11.0
22:31:30  Unable to do partial parsing because saved manifest not found. Starting full parse.
22:31:32  Found 16 models, 13 data tests, 537 macros
22:31:32  
22:31:32  Concurrency: 3 threads (target='dev')
22:31:32  
22:31:35  1 of 16 START sql table model ecommerce_stg.dim_customers_stg .................. [RUN]
22:31:35  2 of 16 START sql table model ecommerce_stg.dim_orders_stg ..................... [RUN]
22:31:35  3 of 16 START sql table model ecomme